In [ ]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install Pandas
import PyPDF2
import pandas as pd
import re
import glob

In [ ]:
data = {}
df = pd.DataFrame()

def extract_info(file):
    main_pdf = PyPDF2.PdfFileReader(file, 'rb')
    main_str = str()

    # decrypt pdf with "" as password
    if main_pdf.isEncrypted:
        main_pdf.decrypt("")

    for pages in range(main_pdf.getNumPages()):
        p = main_pdf.getPage(pages)
        p_text= p.extractText()
        p_lines=str(p_text.splitlines())
        
        if pages == 0:
            main_str += p_lines
        else:
            main_str += " \n "
            main_str += p_lines
    
    # Create dictionary to convert to dataframe later
    data = {
        # Change names if ASX changes their format
        'File Name' : file,
        'Entity Name' : (main_str.split("Entity name")[-1].split("Security on which the Distribution")[0]),
        'Ticker' : (main_str.split("Security on which the Distribution will be paid")[-1])[:3],
        'Record Date' : (main_str.split("2A.4 +Record Date")[-1].split("2A.5 Ex Date")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0]),
        'Payment Date' : (main_str.split("2A.6 Payment Date")[-1].split("2A.7 Are any of the")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0])
        }

        #"Unfranked Amount" : (main_str.split("foreignincome amount")[-1])[4:14],
        #"Imputation Credits" : (main_str.split("foreignincome amount"))
        
    main_str = main_str.replace(" ", "")
    main_str = main_str.replace("Notificationofdividend/distributionNotificationofdividend/distribution", "")
    
    # Add Primary Currency
    if ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22])[0]) == ")":
        data["Primary Currency"] = (main_str.split("Currencyinwhichthedividend/distributionismade")[-1][20:23])
    else:
        data["Primary Currency"] = ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22]))
        
    # Add Dividend per Share and sanity check
    data["Dividend per Share"] = ((main_str.split("notifiedinthisform")[-1][3:].split(".")[0]) + "." +
          (main_str.split("notifiedinthisform")[-1][4:].split(".")[1][0:8]))
    
    try:
        temp = float(data["Dividend per Share"])
    except:
        data.update({"Dividend per Share" : "ERROR GO FIND IT YOURSELF"})
        
    # update with whether dividends per share is actual or estimated
    if (main_str.split("estimatedatthistime?")[-1][:1]) == "Y":
        data.update({"Dividend per Share Status" : "Estimated"})
    else:
        data.update({"Dividend per Share Status" : "Actual"})
    
    # Add Franked + Unfranked info
    data.update({"Franked Amount" : ((main_str.split("3A.5")[0])[-11:])})
    
    try:
        temp = float(data["Franked Amount"])
    except:
        data.update({"Franked Amount" : ((main_str.split("3A.5")[0])[-10:])})
        try:
            temp = float(data["Franked Amount"])
        except:
            data.update({"Franked Amount" : "ERROR GO FIND IT YOURSELF"})
    
    data.update({"Unfranked Amount" : (main_str.split("excludingconduitforeignincomeamount")[-1])[2:13]})
    
    try:
        temp = float(data["Unfranked Amount"])
    except:
        data.update({"Unfranked Amount" : (main_str.split("excludingconduitforeignincomeamount")[-1])[3:13]})
        try:
            temp = float(data["Unfranked Amount"])
        except:
            data.update({"Unfranked Amount" : "ERROR GO FIND IT YOURSELF"})
    
    try:
        if float(data["Unfranked Amount"]) == float(0.0):
            data.update({"Unfranked Amount" : (main_str.split("distributionconduitforeignincomeamountpersecurity")[-1])[3:13]})
    except:
        pass
    
    # Sanity check on dividends (Check if dividends/share = franked + unfranked)
    try:      
        if (float(data["Franked Amount"]) + float(data["Unfranked Amount"])) == float(data["Dividend per Share"]):
            pass
        else:
            data.update({"Unfranked Amount" : (main_str.split("distributionconduitforeignincomeamountpersecurity")[-1])[3:13]})

            if (float(data["Franked Amount"]) + float(data["Unfranked Amount"])) != float(data["Dividend per Share"]):
                data.update({"Franked Amount" : "ERROR GO FIND IT YOURSELF"})
                data.update({"Unfranked Amount" : "ERROR GO FIND IT YOURSELF"})
    except:
            data.update({"Franked Amount" : "ERROR GO FIND IT YOURSELF"})
            data.update({"Unfranked Amount" : "ERROR GO FIND IT YOURSELF"})

    # Update with imputation credits
    if int(main_str.find("Part3E-Other-")) == int(-1):
        data["Imputation Credits"] = float(0)
    else:
        try:
            credits = str(main_str.split("ImputationCreditof")[1].split("perordinaryshare")[0])
            data["Imputation Credits"] = credits
        except:
            data["Imputation Credits"] = float(0)
            
    temp = pd.DataFrame.from_dict(data, orient='index', columns = [""]) 
    return temp

In [ ]:
counter = 0
for file in glob.glob("*.pdf"):
    counter += 1
    print("Extracting File (", counter, "/", len(glob.glob("*.pdf")), ") - ", file)
    df = pd.concat([df, extract_info(file)],axis=1, sort=False)

df.to_csv('output.csv')
print("Saved to CSV!")

In [8]:
main_pdf = PyPDF2.PdfFileReader(file, 'rb')
main_str = str()

# decrypt pdf with "" as password
if main_pdf.isEncrypted:
    main_pdf.decrypt("")

for pages in range(main_pdf.getNumPages()):
    p = main_pdf.getPage(pages)
    p_text= p.extractText()
    p_lines=str(p_text.splitlines())

    if pages == 0:
        main_str += p_lines
    else:
        main_str += " \n "
        main_str += p_lines

# Create dictionary to convert to dataframe later
data = {
    # Change names if ASX changes their format
    'File Name' : file}

    #"Unfranked Amount" : (main_str.split("foreignincome amount")[-1])[4:14],
    #"Imputation Credits" : (main_str.split("foreignincome amount"))

main_str = main_str.replace(" ", "")
main_str = main_str.replace("Notificationofdividend/distributionNotificationofdividend/distribution", "")

print(main_str)

try:
    temp = float(data["Dividend per Share"])
except:
    data.update({"Dividend per Share" : "ERROR GO FIND IT YOURSELF"})

# update with whether dividends per share is actual or estimated
if (main_str.split("estimatedatthistime?")[-1][:1]) == "Y":
    data.update({"Dividend per Share Status" : "Estimated"})
else:
    data.update({"Dividend per Share Status" : "Actual"})

# Add Franked + Unfranked info
data.update({"Franked Amount" : float(main_str.split("distributionthatisfranked")[1].split("%")[0])/100})

try:
    temp = float(data["Franked Amount"])
except:
    data.update({"Franked Amount" : ((main_str.split("3A.5")[0])[-10:])})
    try:
        temp = float(data["Franked Amount"])
    except:
        data.update({"Franked Amount" : "ERROR GO FIND IT YOURSELF"})

data.update({"Unfranked Amount" : (main_str.split("excludingconduitforeignincomeamount")[-1])[2:13]})

try:
    temp = float(data["Unfranked Amount"])
except:
    data.update({"Unfranked Amount" : (main_str.split("excludingconduitforeignincomeamount")[-1])[3:13]})
    try:
        temp = float(data["Unfranked Amount"])
    except:
        data.update({"Unfranked Amount" : "ERROR GO FIND IT YOURSELF"})

try:
    if float(data["Unfranked Amount"]) == float(0.0):
        data.update({"Unfranked Amount" : (main_str.split("distributionconduitforeignincomeamountpersecurity")[-1])[3:13]})
except:
    pass

['1/4UpdateSummaryEntitynameAGLENERGYLIMITED.SecurityonwhichtheDistributionwillbepaidAGL-ORDINARYFULLYPAIDAnnouncementTypeUpdatetopreviousannouncementDateofthisannouncementTuesdayMarch14,2017ReasonfortheUpdateUpdatetoPart4A.6-DRPPriceRefertobelowforfulldetailsoftheannouncementAnnouncementDetailsPart1-Entityandannouncementdetails1.1Nameof+EntityAGLENERGYLIMITED.1.2RegisteredNumberTypeABNRegistrationNumber741150613751.3ASXissuercodeAGL1.4TheannouncementisUpdate/amendmenttopreviousannouncement1.4aReasonforupdatetoapreviousannouncementUpdatetoPart4A.6-DRPPrice1.4bDateofpreviousannouncement(s)tothisupdateThursdayFebruary9,20171.5DateofthisannouncementTuesdayMarch14,20171.6ASX+SecurityCodeAGLNotificationofdividend/distribution']
['2/4ASX+SecurityDescriptionORDINARYFULLYPAIDPart2A-Alldividends/distributionsbasicdetails2A.1Typeofdividend/distributionOrdinary2A.2TheDividend/distribution:relatestoaperiodofsixmonths2A.3Thedividend/distributionrelatestothefinancialreportingorpaymentperiodendingend

In [16]:
float(main_str.split("distributionthatisfranked")[1].split("%")[0])/100

0.8